# Pseudocode

```text
FUNCTION load_tsp_data(name_file, distance_file):
    distance_matrix = []
    READ in name_file
    READ in distance_file
    APPEND distances to distance_matrix

FUNCTION fitness(route, distance_matrix):
    total_distance = 0
    FOR i from 0 to length(route) - 1:
        total_distance += total_distance + distance_matrix[route[i]][route[i+1]]
    total_distance = total_distance + distance_matrix[route[-1]][route[0]]

    RETURN total_distance

FUNCTION mutate(parent_route):
    copy_route = make a copy of the route
    i, j = randomly select 2 different indices from copy_route
    SWAP i,j

    RETURN copy_route

FUNCTION tsp_evolutionary(name_file, distance_file):
    city_names = list of city names from name_file
    distance_matrix = list of distances from distance_file
    best_route = [0,1,2,3,...,n-1]
    best_distance = fitness(best_route,distance_matrix)
    Print "Initial Route Distance:", best_distance, best_route

    same_count = 0
    generation = 1
    WHILE same_count < 3:

        parent_route = best_route
        parent_distance = best_distance

        improvement_found = FALSE
        best_offspring = parent_route
        best_offspring_distance = parent_distance

            FOR offspring_number from 1 to 50:
                offspring = mutate(best_route)
                offspring_distance = fitness(offspring,distance_matrix)

                IF offspring_distance < best_distance:
                    best_route = offspring
                    best_distance = offspring_distance
                    improvement_found = TRUE
            PRINT "Generation:", generation, "Best Distance:", best_distance, "Route:", best_route

            IF improvement_found == TRUE
                same_count = 0
            ELSE:
                same_count += 1
                IF same_count < 3:
                    FOR k from 1 to 8:
                        best_route = mutate(best_route)
            generation += 1

    PRINT "Terminated after", generation, "generations."
    PRINT "Final Best Route Distance:", best_distance
    RETURN best_route
            

In [ ]:
import random

In [ ]:
def load_tsp_data(name_file, distance_file):
    """
    Load tsp city names and distance matrrix from files

    Parameters 
    name_file: text file including city names
    distance_file: text file including distances between cities

    Returns
    city_names: list of city names
    distance_matrix = list of distances between cities
    """

    # Load city names into a list 
    with open(name_file, "r") as f:
        city_names = [line.strip() for line in f if line.strip()]

    # Create empty list 
    distance_matrix = []
    with open(distance_file, "r") as f:
        for line in f:
            if line.strip():
                # split the line by spaces and convert each value to float
                distance_matrix.append([float(x) for x in line.strip().split()])

    # Return both lists for other functions to use
    return city_names, distance_matrix

In [ ]:
def fitness(route,distance_matrix):
    """
    Calculate total distance of a route 

    Parameters
    route: list of integers
    distance_matrix: list 
    
    Returns
    total_distance: float of the total travel distance
    """
    total_distance = 0
    # Add distance between each pair of consecutive cities
    for i in range(len(route)-1):
        total_distance += distance_matrix[route[i]][route[i + 1]]
    # Add distance returning from final city back to starting city
    total_distance += distance_matrix[route[-1]][route[0]]
    
    return total_distance

In [ ]:
def mutate(parent_route):
    """
    Create an offspring route by swapping two cities in a copy of the parent

    Paremeters
    parent_route: list of integers, current best route

    Returns
    new_route: list of integers, new route created by swapping two random cities
    """

    # Make copy so original parent_route isn't changed
    new_route = parent_route.copy()

    # Randomly select two positions in the route to swap
    i,j = random.sample(range(len(parent_route)), 2)

    # Swap the cities at the two randomly select postions
    new_route[i], new_route[j] = new_route[j], new_route[i]
    
    return new_route

In [ ]:
def tsp_evolution(name_file, distance_file):
    """
    Solve the TSP using evolutionary

    Parameters
    name_file: file containing name of cities
    distance_file: file containing distances between cities

    Prints
    Intial Route, the initial route between the cities 
    Generation, the best route found in that generation as city names, and total distance
    Final Route, the best route found within each generation as city names, and total distance

    Returns
    overall_best_route: the best route found within the generations as indexes.
    """

    # Load city names and distance data
    city_names, distance_matrix = load_tsp_data(name_file, distance_file)
    num_cities = len(city_names)

    # Starting route in city order
    best_route = list(range(num_cities))
    best_distance = fitness(best_route, distance_matrix)

    # Tracks the best route found across all generations
    overall_best_route = best_route.copy()
    overall_best_distance = best_distance

    # Print intial route using city names
    initial_route_names = [city_names[i] for i in best_route]
    print("Initial Route Distance:", round(best_distance, 2), initial_route_names)

    same_count = 0 # how many generations had no improvement
    generation = 1 # generation counter
    max_generations = 1000 # max number of generations allowed

    # Evolve until stagnation or maxx generations is reached
    while same_count < 3 and generation <= max_generations:

        # Use the current best route as parent route
        parent_route = best_route
        parent_distance = best_distance

        improvement_found = False
        best_offspring = parent_route
        best_offspring_distance = parent_distance

        num_offspring = 1000 # number of offspring per generation

        # Create offspring and record the best one
        for i in range(num_offspring):
            offspring = mutate(parent_route)
            offspring_distance = fitness(offspring, distance_matrix)

            if offspring_distance < best_offspring_distance:
                best_offspring = offspring
                best_offspring_distance = offspring_distance
                improvement_found = True
        
        # Print best route of the generation in names
        best_offspring_names = [city_names[i] for i in best_offspring]
        print(f"Generation {generation}: {best_offspring_names} | Distance = {round(best_offspring_distance, 2)}")

        # Update parent route if improvement ocurred
        if improvement_found:
            best_route = best_offspring
            best_distance = best_offspring_distance
            same_count = 0

            # Update global best 
            if best_distance < overall_best_distance:
                overall_best_route = best_route.copy()
                overall_best_distance = best_distance
        else:
            # No improvement found
            same_count += 1
            if same_count < 3:
                print("No improvement")

                # Force mutate the route 5 times to escape local minimum
                for i in range(5):
                    best_route = mutate(best_route)
                best_distance = fitness(best_route,distance_matrix)

        generation += 1

    # Convert best final route to city names
    route_names = [city_names[i] for i in best_route]

    # Print results
    print("\nTerminated after", generation, "generations.")
    print("Final Best Route:", route_names)
    print("Final Distance:", round(overall_best_distance, 2))

    return overall_best_route



In [ ]:
tsp_evolution("thirty_cities_names.txt", "thirty_cities_dist.txt")
